In [1]:
import tensorflow as tf

In [2]:
import numpy as np
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [3]:
import io,os,sys,types
from IPython import get_ipython #for import notebook
from nbformat import read #for import notebook
from IPython.core.interactiveshell import InteractiveShell #for import notebook

In [4]:
def find_notebook(fullname, path=None): #for import notebook
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [5]:
class NotebookLoader(object): #for import notebook
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [6]:
sys.meta_path.append(NotebookFinder())

In [7]:
import myprepdata

importing Jupyter notebook from myprepdata.ipynb


In [8]:
import encoder

importing Jupyter notebook from encoder.ipynb


In [9]:
import decoder

importing Jupyter notebook from decoder.ipynb


In [10]:
dataset_loader = myprepdata.prep()

[  8.89466166e-01   2.06377721e+00   6.17529333e-01   3.18206835e+00
   7.67453134e-01  -1.99592626e+00  -1.48355320e-01  -1.96766481e-01
  -3.16980266e-04   2.14467272e-01  -4.14004624e-01   3.57001126e-01
  -1.75457168e+00  -1.39167953e+00   2.63751793e+00   1.19258475e+00
   6.61663294e-01  -3.63710284e-01   1.10500908e+00  -1.90649366e+00
  -1.27631116e+00   7.06363976e-01   2.16604018e+00   7.20149457e-01
   1.19366109e+00   3.15650672e-01  -3.05436397e+00   7.40569711e-01
   8.19141865e-01   1.18011284e+00   4.69861925e-01   8.27771947e-02
   1.12338221e+00   2.47764182e+00   3.43266904e-01  -2.68325806e+00
  -1.40979004e+00  -1.32848775e+00   7.03199029e-01   9.64148343e-02
   3.22712660e-01  -2.13381246e-01  -1.01692128e+00   1.25775146e+00
   1.02071427e-01  -1.11394691e+00  -2.41053867e+00  -1.19636559e+00
   4.32757884e-01   1.38155115e+00   1.13593802e-01  -1.06976998e+00
   1.57687080e+00   1.43941748e+00   2.68036485e-01   1.28006709e+00
   1.21146309e+00   6.14654839e-01

In [11]:
newline = dataset_loader.prepare_line()[0]
newline

'Res@@ um@@ ption of the session'

In [12]:
deline = dataset_loader.de_prepare_line()[0]
deline

'Wiederaufnahme der Sitzungsperiode'

In [13]:
#dataset_loader.get_wvmodel().wv["Res@@"] #good

In [14]:
#len(newline.split(' ')) #6

In [15]:
#newline.split(' ')[0] #'Res@@'

In [16]:
def make_length_np(length):
    i = 0
    leng = np.array([])
    
    while(i < length):
        leng = np.append(leng,[1])
        i = i + 1
        
    return leng



In [17]:
def make_pad_line(dataset_loader,input_or_output,linenum):
    if input_or_output == 0:
        newline = dataset_loader.prepare_line()[linenum]
        padding_line = np.full((1, 484, 100), 0, dtype="float32")
        i = 0
        while(i < len(newline.split(' '))):
            padding_line[0][i] = dataset_loader.get_wvmodel().wv[newline.split(' ')[i]]
            i = i + 1
    else:
        newline = dataset_loader.de_prepare_line()[linenum]
        padding_line = np.full((1, len(newline.split(' ')), 100), 0, dtype="float32")
        i = 0
        while(i < len(newline.split(' '))):
            padding_line[0][i] = dataset_loader.de_get_wvmodel().wv[newline.split(' ')[i]]
            i = i + 1
            
    return newline, padding_line

In [18]:

with tf.Graph().as_default(): #set default graph
    
    
    newline, padding_line = make_pad_line(dataset_loader,0,0)
    deline, de_padding_line = make_pad_line(dataset_loader,1,0)
    
    
    encoder1 = encoder.naive_encoder(layers=1, directions =1, maxv=484, hiddensizes=100, bach_s=1)
    decoder1 = decoder.attention_decoder(layers = 1, directions = 1, hiddensize = 100, input_length = 484, 
                       teacher_forcing = False)
    
    #workers, _ = tf.test.create_local_cluster(1, 0) #python 3.6 <
    #worker = workers[0] #python 3.6 < 


    with tf.Session() as sess:
            
        encoder1.hidden_init(sess)
        decoder1.hidden_init(sess)
        
        print(np.shape(padding_line))
        en_out = encoder1.getoutput(sess,padding_line) #encoder_decoder seperate training ver
        en_state = encoder1.getstate(sess,padding_line) #encoder_decoder seperate training ver
        #en_out = encoder1.getoutput()
        #en_state = encoder1.getstate()
        
        inputdic = {'sess':sess,
                'realsente':de_padding_line[0], 
                'output_length': make_length_np(len(deline.split(' '))),
                    'input_v':en_out[0][0],'enstate':en_state[0]}
                   #'line':padding_line}
        
        
        t = 0
        while(t < 50):
            
            decoder1.calcost(**inputdic)
            decoder1.training(**inputdic) #why decoder only training....
            
            t = t + 1

(1, 484, 100)
[3.1680763]
[None]
[3.0666564]
[None]
[2.3715637]
[None]
[2.1503482]
[None]
[2.0489268]
[None]
[1.9277451]
[None]
[1.8415226]
[None]
[1.82366]
[None]
[1.8119841]
[None]
[1.8016429]
[None]
[1.8115758]
[None]
[1.8053695]
[None]
[1.8032975]
[None]
[1.8032998]
[None]
[1.805246]
[None]
[1.8029969]
[None]
[1.8011111]
[None]
[1.7926632]
[None]
[1.798755]
[None]
[1.7944866]
[None]
[1.795181]
[None]
[1.7902201]
[None]
[1.7862146]
[None]
[1.7866559]
[None]
[1.7854223]
[None]
[1.7841773]
[None]
[1.7863191]
[None]
[1.7846357]
[None]
[1.7852346]
[None]
[1.7860024]
[None]
[1.7847495]
[None]
[1.7846185]
[None]
[1.7846429]
[None]
[1.7841489]
[None]
[1.7846895]
[None]
[1.7841376]
[None]
[1.7839479]
[None]
[1.7843113]
[None]
[1.7840743]
[None]
[1.7838371]
[None]
[1.7837527]
[None]
[1.7829866]
[None]
[1.7815353]
[None]
[1.7826532]
[None]
[1.7829386]
[None]
[1.7827652]
[None]
[1.7821074]
[None]
[1.7816671]
[None]
[1.7821903]
[None]
[1.7820805]
[None]


In [19]:
#import portpicker